In [1]:
import pretty_midi
from corr_mat import *
from track import Track
from util import get_instrument_type, create_loop_dict
import os
from tqdm import tqdm
import time
import csv

In [2]:
test_file = "./test_midi.mid"
pm = pretty_midi.PrettyMIDI(test_file)

In [3]:
final_loops = []
for idx, instrument in enumerate(pm.instruments):
    instrument_type = get_instrument_type(instrument)
    track = Track(pm, instrument)
    note_list = track.notes
    lead_mat, lead_dur = calc_correlation(note_list)
    loops, loop_endpoints = get_valid_loops(track, lead_mat, lead_dur)
    print(f"found {len(loops)} loops in {instrument}")

    for endpoint in loop_endpoints:
        start, end, beats, density = endpoint
        time_sig = track.get_time_sig_at_time(start)
        print(idx, start, end, time_sig, beats, instrument_type, density)
    for loop in loops:
        final_loops.append(loop)
print(f"{len(final_loops)} total loops in {len(pm.instruments)} tracks")

found 1 loops in Instrument(program=0, is_drum=True, name="GOT TO HAVE YOUR LOVE     ")
0 18.113216000000005 22.641519999999996 (4, 4) 8.0 DRUM 3.875
found 1 loops in Instrument(program=39, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
1 9.056607999999999 13.584912000000005 (4, 4) 8.0 BASS 2.0
found 1 loops in Instrument(program=4, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
2 33.96227999999998 43.01888799999996 (4, 4) 16.0 PIANO 0.5625
found 1 loops in Instrument(program=48, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
3 24.905671999999992 29.433975999999983 (4, 4) 8.0 ENSEMBLE 0.75
found 1 loops in Instrument(program=48, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
4 24.905671999999992 29.433975999999983 (4, 4) 8.0 ENSEMBLE 0.75
found 1 loops in Instrument(program=28, is_drum=False, name="GOT TO HAVE YOUR LOVE     ")
5 27.169823999999988 31.69812799999998 (4, 4) 8.0 GUITAR 1.75
found 0 loops in Instrument(program=2, is_drum=False, name="GOT TO HAVE YOUR LOVE     "

In [4]:
def run_file(file_path, name):
    try:
        pm = pretty_midi.PrettyMIDI(file_path)
    except:
        #print(f"failed to parse {file_path}, skipping")
        return 0,0,[]
    
    total_loops = 0
    loops = []
    for idx, instrument in enumerate(pm.instruments):
        instrument_type = get_instrument_type(instrument)
        track = Track(pm, instrument)
        note_list = track.notes
        lead_mat, lead_dur = calc_correlation(note_list)
        _, loop_endpoints = get_valid_loops(track, lead_mat, lead_dur)
        for endpoint in loop_endpoints:
            time_sig = track.get_time_sig_at_time(start)
            if time_sig is None:
                continue
            loop_dict = create_loop_dict(endpoint, idx, instrument_type, time_sig, name)
            loops.append(loop_dict)
        total_loops += len(loop_endpoints)
    return total_loops, len(pm.instruments), loops

In [5]:
directory_main = "D:\\Documents\\GigaMIDI\\non_express"
directory_inner = "Train-80%-Non-Expressive\\GigaMIDI-Train-Music-No-Drum-MD5-Non-Expressive\\"
full_directory = os.path.join(directory_main, directory_inner)

In [6]:
len(os.listdir(full_directory))

143602

In [7]:
total_loops = 0
total_tracks = 0
all_loops = []
for file in tqdm(os.listdir(full_directory)):
    #start = time.time()
    full_path = os.path.join(full_directory, file)
    string_path = os.path.join(directory_inner, file)
    num_loops, num_tracks, loops = run_file(full_path, string_path)
    total_loops += num_loops
    total_tracks += num_tracks
    for loop in loops:
        if len(loop) > 0:
            all_loops.append(loop)
    #end = time.time()
    #print(f"{file}: {num_tracks} tracks {num_loops} loops {round(end - start,2)}")
print(f"Found {total_loops} loops in {total_tracks} tracks across {len(os.listdir(full_directory))} files")

import csv
keys = all_loops[0].keys()
with open('loop_data.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_loops)

  0%|          | 14/143602 [00:01<2:22:05, 16.84it/s]

failed to parse D:\Documents\GigaMIDI\non_express\Train-80%-Non-Expressive\GigaMIDI-Train-Music-No-Drum-MD5-Non-Expressive\00019053cd6b4ec7472dfecf56e66300.mid, skipping
failed to parse D:\Documents\GigaMIDI\non_express\Train-80%-Non-Expressive\GigaMIDI-Train-Music-No-Drum-MD5-Non-Expressive\0001a7bd02560ca952eb2a19053b5ca5.mid, skipping


  0%|          | 23/143602 [00:01<1:34:53, 25.22it/s]d:\Documents\GigaMIDI\midi_loop_detection\.venv\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
  0%|          | 59/143602 [00:22<7:57:10,  5.01it/s] 

failed to parse D:\Documents\GigaMIDI\non_express\Train-80%-Non-Expressive\GigaMIDI-Train-Music-No-Drum-MD5-Non-Expressive\001660fdc400a73feb81c235c215604a.mid, skipping


  0%|          | 71/143602 [00:23<3:46:01, 10.58it/s]

...

 89%|████████▊ | 127202/143602 [9:17:42<1:40:50,  2.71it/s]

failed to parse D:\Documents\GigaMIDI\non_express\Train-80%-Non-Expressive\GigaMIDI-Train-Music-No-Drum-MD5-Non-Expressive\e298cc62ae91bdb2591ba8fb6eb65761.mid, skipping


 89%|████████▊ | 127221/143602 [9:17:45<41:28,  6.58it/s]  

failed to parse D:\Documents\GigaMIDI\non_express\Train-80%-Non-Expressive\GigaMIDI-Train-Music-No-Drum-MD5-Non-Expressive\e2a2ee2e5e7f423d26ec5434cc3fa0ee.mid, skipping


 89%|████████▊ | 127249/143602 [9:17:59<3:02:15,  1.50it/s]